# Quickstart

In [1]:
from foxflow.reader import BagfileReader
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Foxglove API key (set in .env)
FOXGLOVE_KEY = os.getenv("FOXGLOVE_KEY")

In [3]:
r = BagfileReader(FOXGLOVE_KEY)
r.print_recordings()

Available Recordings:

bc_pitstop                               → rec_0dyFsZgY0g4uRecO
bc_hm_right                              → rec_0dyG6uEXMKleoJ1z
bc_hm_left                               → rec_0dyG7RYeZx4Zmyiw
runde_vdi_lausitz                        → rec_0dyVbk9iNjsByd1Z
vdi_adc_2025_left                        → rec_0dyVch3zRMAviL5U
vdi_adc_2025_right                       → rec_0dyVcoxClnfxzloX
szechenyi_bc_ir_60_left_0                → rec_0e02rpVN69OXAheD
szechenyi_bc_ir_90_left_0                → rec_0e02rqfrxfbW3xKy
szechenyi_bc_uss_left                    → rec_0e02rcEt7K9TwwR4
testfile                                 → rec_0e4fjnBMpmFLl9Np
bc_acc_left_I                            → rec_0dxlVpedvsMyiMmM
bc_acc_left_lidar                        → rec_0dxlWSHulsX1fuoR
bc_acc_right_I                           → rec_0dxlWmQlvwikncqG
bc_acc_right_lidar                       → rec_0dxlYIDbBRQIpBFt
bc_left_vol_II                           → rec_0dxlYUL1DQxE1MGg
bc_left_vol_I    

In [9]:
df_info = r.select_recording_by_name('testfile')
display(df_info)

,topic,version,encoding,schema_encoding,schema_name
0,/camera/camera/color/image_raw,2b4894e8c31a09b9a1f7f9d91a9b5e1b,cdr,ros2msg,sensor_msgs/msg/Image
1,/rc/ackermann_cmd,e31eb8800d99f34ad9cf4113bc497ad8,cdr,ros2msg,ackermann_msgs/msg/AckermannDriveStamped


#### Laser Scan

In [ ]:
df_info = r.select_recording_by_name('parking_spot_scan_boxes')
df_scan = r.read_topic('/scan')[0]
df_scan.head()

,timestamp_ns,angle_min,angle_max,angle_increment,time_increment,scan_time,range_min,range_max,ranges,intensities
0,946690865515522784,-3.124139,3.141593,0.003483,0.000049,0.088427,0.15,16.0,"[0.9120000004768372, 0.9120000004768372, 0.913...","[0.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0..."
1,946690865605088096,-3.124139,3.141593,0.003483,0.000051,0.091301,0.15,16.0,"[0.9100000262260437, 0.9120000004768372, 0.912...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."
2,946690865697576512,-3.124139,3.141593,0.003483,0.000050,0.090358,0.15,16.0,"[0.9100000262260437, 0.9100000262260437, 0.910...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."
3,946690865789197120,-3.124139,3.141593,0.003483,0.000054,0.097363,0.15,16.0,"[0.9120000004768372, 0.9120000004768372, 0.910...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."
4,946690865887939680,-3.124139,3.141593,0.003483,0.000051,0.090873,0.15,16.0,"[0.9100000262260437, 0.9079999923706055, 0.906...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."


In [ ]:
row = df_scan.iloc[0]

# ranges
rng = np.array(row["ranges"])

# angles (rad → deg)
theta = np.degrees(
    np.linspace(
        row["angle_min"],
        row["angle_max"],
        num=len(rng),
        endpoint=False
    )
)

# polar scatter with range as color
fig = px.scatter_polar(
    r=rng,
    theta=theta,
    color=rng,                 # distance → color
    color_continuous_scale="Viridis",
    labels={"color": "range [m]"},
)

fig.show()


#### Camera

In [ ]:
df_info = r.select_recording_by_name('testfile')
df_cam, images = r.read_topic('/camera/camera/color/image_raw', return_images=True)

,timestamp_ns
0,1760466160755695557
1,1760466160755695557
2,1760466160922628662
3,1760466160922628662
4,1760466161089579102


In [16]:
fig = px.imshow(images[0])
fig.show()

#### NavSatFix

In [8]:
df_info = r.select_recording_by_name('rtb')
df_sat = r.read_topic('/sensor_stack/adma/fix')

In [9]:
fig = px.scatter_geo(
    df_sat,
    lat="latitude_deg",
    lon="longitude_deg",
)

fig.show()


#### PointCloud2

In [7]:
df_info = r.select_recording_by_name('rtb')
df_pc = r.read_topic('/sensor_stack/lidars/lid_1/livox/lidar_192_168_1_150')
df_sat = r.read_topic('/sensor_stack/adma/fix')

In [10]:
pts = df_pc.loc[0, "points"]          # (N, 4): x, y, z, intensity

fig = go.Figure(
    go.Scatter3d(
        x=pts[:, 0],
        y=pts[:, 1],
        z=pts[:, 2],
        mode="markers",
        marker=dict(
            size=2,
            color=pts[:, 3],          # intensity
            colorscale="Viridis",
            showscale=True,
        ),
    )
)

fig.show()


#### Events

In [20]:
# Fetch events (list of dicts)
events = r.get_events()

# Flatten nested dicts into a table
df = pd.json_normalize(events)

# Simple timeline: one bar per event
fig = px.timeline(
    df,
    x_start="start",   # event start time
    x_end="end",       # event end time
    y="id",            # one row per event
    hover_data=["id"], # minimal hover info
)

# Display plot
fig.show()
